In [1]:
from utils import *
from sknn_scipy import *
import pandas as pd
import numpy as np
import time
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import norm
from tqdm import tqdm

In [2]:
FOLDER_TRAIN = 'data/train/'
FOLDER_TEST = 'data/test/'
train, test = load_dataset(FOLDER_TRAIN, FOLDER_TEST, feather=True)

load_dataset
 -- loaded in: 2.976196527481079s
 -- filtered in: 11.245805025100708s
 -- actions:  66346428
 -- items:  2262292
 -- lists:  1000000


In [3]:
OUTPUT_FOLDER = 'output/'
rec = SessionKNN(2000, 0, sampling='else', idf_weight=1, folder=OUTPUT_FOLDER)

In [4]:
start = time.time()
rec.train(train, test)
end = time.time()
print('Training Time: {}'.format(end - start))

Matrix build time:  12.622575044631958
IDF calculate time:  0.36583971977233887
Training Time: 13.069051742553711


In [5]:
n = []
s = []
p = []
pids = np.random.choice(train['playlists']['playlist_id'].values, 100)
for pid in tqdm(pids):
    name = train['playlists'][train['playlists']['playlist_id'] == pid]['name']
    ntracks = train['playlists'][train['playlists']['playlist_id'] == pid]['num_tracks']
    tracks = train['actions'][train['actions']['playlist_id']==pid]['track_id'].values
    artist_ids = train['actions'][train['actions']['playlist_id']==pid]['artist_id'].values
    start = time.time()
    _, n_time, s_time = rec.predict(name, tracks, playlist_id=pid, artists=artist_ids)
    end = time.time()
    p.append(end - start)
    n.append(n_time)
    s.append(s_time)
print('Mean find neighbor time: ', np.mean(n))
print('Mean score item time: ', np.mean(s))
print('Mean prediction time: ', np.mean(p))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:59<00:00,  1.67it/s]

Mean find neighbor time:  0.4181516695022583
Mean score item time:  0.05046917915344238
Mean prediction time:  0.46971851110458374
